# デュエル中のデータの扱い方についてのサンプルです。

## デュエル中に扱う主なデータ構造

デュエルの進行中に送られてくる情報は以下の3種類のデータがあります。

### 1. `DuelStateData` 
デュエルの現在の情報
### 2. `CommandRequest`
入力要求に関する情報
### 3. `DuelLogData`
デュエルのログの情報

ここではこのそれぞれについて、サンプルのデータを使いながらデータの構造や扱い方についての説明をします。

## 準備

ライブラリのimportや使用するutilのインスタンスを作成します。

In [ ]:
import os
from IPython.display import Markdown, display
from ygo.util.card import CardUtil
from ygo.util.udi_log import UdiLogUtil
from ygo.util.text import TextUtil
from ygo.gui.udi_gui_thread import UdiGUIThread
from ygo.constants.enums import DuelLogType, SelectionType
from ygo.models.duel_log_data import CardHappenData, DuelEndData

In [ ]:
# テキスト取得用インスタンス作成
text_util = TextUtil()
# カード情報取得用インスタンス作成
card_util = CardUtil()

In [ ]:
# データ表示用guiスレッド作成
udi_gui_thread = UdiGUIThread()
udi_gui_thread.start()

以下で使うデータはサンプルのudi_log_dataから確認します。

デュエル中はudi_ioのそれぞれの対応する関数から取得することになります。

In [ ]:
# サンプルのログデータ取得
SAMPLE_DATA_PATH = os.path.join(os.getcwd(), "udi_log_data/udi_log_sample.gz")
udi_log_data = UdiLogUtil.load_udi_log(SAMPLE_DATA_PATH)

## 1. デュエルの現在の情報、```DuelStateData```について

プレイヤーのLP、カードの情報といったデュエルの現在の情報はDuelStateDataに入っています。

DuelStateDataには、

- カードによらない情報（general_data）
- カード毎の情報（duel_card_table）
- チェーンに関する情報（chain_stack）

の3つの情報があります。

### サンプルデータの取得
udi_log_dataの中からDuelStateDataを1つ取得してみます。

デュエル中は```udi_io.get_duel_state_data()```で取得してください。

In [ ]:
# 例としていいデータを探しています
sample = udi_log_data[-1]
for entry in udi_log_data:
    if len(entry.duel_state_data.chain_stack) > 2:
        sample = entry
        break

# duel_state_dataを取得
duel_state_data = sample.duel_state_data

# guiで表示します
udi_gui_thread.set_data(sample.duel_log_data, sample.command_request, sample.duel_state_data)

### カードによらないデュエルの情報
プレイヤーのLPなどカードによらないデュエルの情報は```duel_state_data.general_data```に入っています。

lpやsummon_numといった配列になっているものは[自分、相手]の順で入っています。

データの取得方法は下記のセルを参照してください

In [ ]:
general_data = duel_state_data.general_data 
# 表にしてみてみます。
display(Markdown(text_util.get_general_data_markdown(general_data)))

print(general_data) # そのまま
print(f"{general_data.current_phase=}") # いまのフェイズを取得

### カード毎の情報
デュエル中の盤面情報に相当するカード毎の情報は```duel_state_data.duel_card_table```に入っています。このデータはどの場所にどのカードが存在するかというような場所を基準とした構造ではなく、どのカードがどこに存在するかという**カードを基準とした構造**となっていることが特徴です。

`duel_card_table`は、デッキ、EXデッキ、生成されたトークンを含む**自分のカード100枚分+相手のカード100枚分**の計200の要素を持つ配列です。各要素は`DuelCard`というデータクラスとなっており、カードの位置情報などが格納されています。

イメージをつかむためにまずは下記のセルでサンプルデータの全体を表にして見てみます。（出力された表ではカードが存在しない行は取り除かれています。）

In [ ]:
duel_card_table = duel_state_data.duel_card_table
# duel_card_tableをマークダウン形式で取得する
markdown_text = text_util.get_duel_card_table_markdown(duel_card_table)
# 表示
display(Markdown(markdown_text))

#GUIで状況を確認
udi_gui_thread.set_data(sample.duel_log_data, sample.command_request, sample.duel_state_data)

UDIから送られてくる盤面情報はこのような表の形で表すことができます。（ここでのtable_indexはduel_card_table配列でのインデックスです。）

各列はDuelCardのフィールドになっているので、意味についてはDuelCardの定義を参照してください。

今回はお互い40枚のデッキでEXデッキはなくトークンもないので、0から39の行に自分の使用デッキの各カードの情報が入っており、相手のカードの情報は100から139の行に入っています。

ここで、自分のカードと相手のカードについて、以下の違いがあることに注意してください。

- 0から39までの自分のカードのcard_idには常に値が存在し、そのtable_indexはカード一枚一枚に対応したユニークな値となっている。
- 100から139の相手のカードについては、カードが判明した段階で裏側のカードのcard_idの値が埋まっていく。一度デッキに戻るなどして不明となったカードの行に改めて判明したカードが割り当てられる場合があるので、相手側のtable_indexはカード一枚一枚に対応した値ではない。

GUIが表示できている場合は、table_index=3の自分のメインモンスターゾーン中央の「青眼の白龍」など、盤面の見た目とこの表の値が対応していることを確認してみてください。GUI盤面の表示はこのduel_card_tableによって作成されています。

pythonでのデータの取得方法については下記のセルの例を参照してください。

In [ ]:
# 同様にtableを取得
duel_card_table = duel_state_data.duel_card_table

# 長さ200の配列であることを確認してみます
print(f"{len(duel_card_table)=}")

# デュエルに存在するカードの情報が各cardに入っています
card = duel_card_table[0] # 試しに0番目のカードの情報を見てみます
print(f"{card=}") # そのままデータクラスを表示してみます
print(f"{card.card_id=}, 「{text_util.get_card_name(card.card_id)}」") # 個別の情報を取得見てみます

# text_utilで数値が0以上の意味のある値が入っている要素のみを表示してみます
print(text_util.get_duel_card_text(card))

# この行のカード名を取得してみます
card_name = text_util.get_card_name(card.card_id)

# そのカードの攻撃力を取得してみます
print(f"duel_card_tableから得られた攻撃力{card.atk_val}")

# tableにはフィールドでの攻撃力しか入っていないので、card_utilからカードに記載された攻撃力を取得してみます
if card.atk_val < 0:
    atk_val = card_util.get_atk(card.card_id)
    print(f"「{card_name}」のカードに記載された攻撃力={atk_val}")

### チェーンに関する情報
チェーンに関する情報はduel_state_data.chain_stackに入っています。

チェーンの情報は現在チェーンに積まれているカードや処理中のカードの情報などが含まれています。

表示と情報取得は下記のセルを参照してください。

In [ ]:
chain_stack = duel_state_data.chain_stack

if len(chain_stack) > 0:
    # 中身を表形式で表示
    display(Markdown(text_util.get_chain_stack_markdown(chain_stack)))

    # 試しに一番上のChainDataについて見てみます
    chain_data = chain_stack[-1]

    # チェーンの一番上のカードのcard_idを取得
    top_card_id = chain_data.card_id
    print(f"チェーンの一番上のカード={text_util.get_card_name(top_card_id)}({top_card_id})")

    # 対象のカードもあれば確認できます
    target_table_index_list = chain_data.target_table_index_list
    if len(target_table_index_list):
        for index in target_table_index_list:
            target_card = duel_card_table[index]
            print(text_util.get_duel_card_text(target_card))
else:
    print("チェーン情報無し")


表にあるように、各チェーン情報はカードの効果やチェーンの状況だけではなくduel_card_table上のカードと紐づいています。

例えばこのデータからチェーン1の大嵐が相手の魔法＆罠ゾーン左から2番目で発動していることや、

チェーン2の「月の書」の対象のカードを表すtarget_table_index_listに3が存在しているので、この対象がduel_card_tableのtable_indexが3である「青眼の白龍」であることを先ほどのduel_card_tableから確認してみてください。

## 2. 入力要求に関する情報、```CommandRequest```について

入力要求の情報は、CommandRequestに入っています。

入力要求とは、

- メインフェイズにどのカードを発動するか
- カードの発動に対してどのカードをチェーンするか
- バトルフェイズにどのカードで攻撃するか
- 効果の対象でどのカードを選択するか
- 効果の処理中にどのカードを選択するか

といったような、デュエルの進行中にプレイヤーの選択が求められるときにクライアントから送られてくる情報です。

AIエージェントは入力要求が送られてきた際にはここから現在の選択に関する情報と実行できるコマンドの選択肢を把握し、その選択肢から一つを選んでクライアントに送信してデュエルを進めていくことになります。

### サンプルデータの取得
udi_log_dataの中からCommandRequestを取得してみます。

デュエル中は`udi_io.get_command_request()`で取得してください。

ここではDuelStateDataの例とは別のデータを使います。

In [ ]:
# 1つデータを取得してみます
sample = udi_log_data[-1]

# 例として良いcommand_requestを見つけて代入
for entry in udi_log_data:
    if len(entry.command_request.commands) > 1 and len(entry.command_request.command_log)> 1:
        sample = entry
        break

# 1つcommand_requestがあるデータを取得してみます
command_request = sample.command_request
# 対応するduel_state_dataも参照するためもっておきます。
duel_state_data = sample.duel_state_data

# guiにも表示してみます
udi_gui_thread.set_data(sample.duel_log_data, sample.command_request, sample.duel_state_data)

### どのような選択かの情報
どのような選択かの情報は、主に種類を表すselection_typeと内容を表すselection_idに入っています。

selection_typeにはメインフェイズやチェーンの効果処理中といった種類の情報が入っており、selection_idには効果処理中に選ぶものを指定するなどのテキストの情報が入ります。

In [ ]:
# 選択の種類はselection_typeに入っています
selection_type = command_request.selection_type
print(f"選択の種類（selection_type）: 「{text_util.get_selection_type_text(selection_type)}」")

# 選択の説明テキストはselection_idに入っています（selection_typeによっては入っていないこともあります）
selection_id = command_request.selection_id
print(f"選択の説明（selection_id）: 「{text_util.get_selection_id_text(selection_id)}」")
    

今回の例ではチェーンの効果処理中で、儀式召喚に必要なレベル分のモンスターの選択を求められているようです。

チェーンの効果処理中ということで先ほどのduel_state_dataにあったチェーンに関する情報chain_stackについても参照してみます。

In [ ]:
# chain_stackをduel_state_dataから持ってきます
chain_stack = duel_state_data.chain_stack

# 表で見てみます
display(Markdown(text_util.get_chain_stack_markdown(chain_stack)))

# チェーンの効果処理中なので要素はあるはずですが一応長さを確認
if len(chain_stack) > 0:
    # 効果処理中のカードが知りたいので一番上のカードを見てみます
    chain_top = chain_stack[-1]
    # 効果処理中のカードのcard_idを見てみます
    card_id = chain_top.card_id
    # 取得したcard_idを表示してみます
    print(f"{card_id=}, {text_util.get_card_name(chain_top.card_id)=}")



先ほど儀式召喚に必要なレベル分のモンスターの選択を求められているという状況でしたが、今回は高等儀式術の通常モンスターを墓地へ送る効果処理中だったようです。

### とれる選択肢の情報
その入力要求でのコマンドの選択肢はcommandsに入っています。

commandsにはその選択肢に関する情報CommandEntryが配列で入っており、入力要求が来た時にはこの中から一つを選択してcommands内のインデックスで返答することになります。

まずはcommandsについて、各フィールドを列にし、要素を行とした表にしてみてみます。

In [ ]:
commands = command_request.commands # コマンドの選択肢を取得

display(Markdown(text_util.get_commands_markdown(commands)))

この表のそれぞれの行が1つの選択肢に対応しています。

選択肢に関する情報はこの行にまとまっており、選択肢に関係のある情報のフィールドだけ値が存在することになります。

先ほどselection_typeとselection_idとchain_stackから今回の入力要求は「高等儀式術」のデッキから通常モンスターを墓地へ送る効果処理だと判断していましたが、

commandsにはそれぞれcard_idに自分のデッキ内の別々の通常モンスターカードが並んでいるので、そこを見てデッキの中のカードを決定する必要があるということになります。

pythonでのデータの取得方法については下記のセルの例を参照してください。

In [ ]:
commands = command_request.commands # コマンドの選択肢を取得

# 各commandについて中身を見てみます
for i, command in enumerate(commands):
    print("###########################################################")
    print(command)
    print(text_util.get_command_entry_text(command))
    print(f"commandの種類: {command.command_type}")
    print(f"commandに関連するカードのcardId: {command.card_id}")

    # commandが特定のカードに関連している場合は、table_indexからduel_card_table上のカードも参照できます
    table_index = command.table_index # duel_state_dataのduel_card_table上でのindex
    if table_index > -1:
        card = duel_state_data.duel_card_table[table_index] # tableの情報を取得
        print(f"カードのtableの情報: {text_util.get_duel_card_text(card)}")
        
    print("###########################################################")
    

### 現在の選択に関する過去の自分の選択の情報
現在の選択に関係した今までのコマンドの情報がcommand_request.command_logに入っています。

command_logに含まれるコマンドは基本的には自分がコマンドを実行してから相手に入力要求がわたるまでの一連のコマンドですが、

チェーンの効果処理中では、その効果処理中で既に選択したコマンドや発動するためのコストなどのコマンドがこのcommand_logに含まれます。

また、攻撃宣言が巻き戻され攻撃対象を選びなおした場合は、最初の攻撃宣言でのコマンドもcommand_log含まれます。

例えば先ほどの高等儀式術の例では通常モンスターを選択しようとしていましたが、その前にどの儀式モンスターを選択していたのかといった情報がこのcommand_logから確認できます。

In [ ]:
command_log = command_request.command_log

# まずは表で確認
display(Markdown(text_util.get_command_log_markdown(command_log)))

# データ取得方法
print(command_log)

# コマンドログの確認
if len(command_log) > 0:
    command_log_entry = command_log[-1] # 一番最後の直前のログについて見てみます
    print(f"直前の関連するcommand: {text_util.get_command_entry_text(command_log_entry.command)}") # その時選択したcommand
    print(f"直前の関連するselection_type: {command_log_entry.selection_type}") # その時のselection_type
    print(f"直前の関連するselection_id: {command_log_entry.selection_id}") # その時のselection_id

今回の例ではまずメインフェイズで高等儀式術の効果発動を選択し、特殊召喚するモンスターとして「白竜の聖騎士」を選択していたようです。

### メインフェイズでの例

もう一つメインフェイズでの例も見てみます。

In [ ]:
# メインフェイズでのログデータを見るために探しています
main_phase_sample = udi_log_data[-1]
for entry in udi_log_data:
    if entry.command_request.selection_type == SelectionType.MAIN_PHASE and len(entry.command_request.commands) > 1:
        main_phase_sample = entry
        break

# guiでも見てみます
udi_gui_thread.set_data(main_phase_sample.duel_log_data, main_phase_sample.command_request, main_phase_sample.duel_state_data)

# commandsを表で見てみます
commands = main_phase_sample.command_request.commands
display(Markdown(text_util.get_commands_markdown(commands)))

# tableを表で見てみる場合は下のコメントアウトを外してください
# duel_card_table = main_phase_sample.duel_state_data.duel_card_table
# display(Markdown(text_util.get_duel_card_table_markdown(duel_card_table)))

今回のメインフェイズの例では、各カードに対してセットや召喚といったcommand_typeが存在しています。

table_indexに値がある場合は該当のカードのduel_card_tableでのインデックスと対応しているということなので、
上のセルのtableを表示する部分のコメントアウトを外して実行してduel_card_tableも見てみると対応を確認できるかと思います。

また、index=7の例にあるようにカードとは関係のないフェイズ移行といったコマンドも同様にこの情報で表現されています。

## 3. デュエルのログの情報、```DuelLogData```について

カードの発動、移動、攻撃宣言などデュエル中に起こったことの情報はDuelLogDataに入っています。

DuelStateDataと同様にデュエル中の状況を把握するために使用することができます。

### サンプルデータの取得
udi_log_dataの中からDuelLogDataを1つ取得してみます。

デュエル中は```udi_io.get_duel_log_data()```（デュエル開始からの情報）または```udi_io.get_new_duel_log_data()```（直近のデータで追加された情報）で取得してください。

※今回サンプルデータとして使っている保存されたudi_log_dataについては```udi_io.get_new_duel_log_data()```に対応するものはありません。

In [ ]:
# デュエル終了時のデータを取得してみます
sample = udi_log_data[-1]

# 終了時でのduel_log_dataを取得します（デュエル開始からデュエル終了までのログ）
duel_log_data = sample.duel_log_data

### デュエルのログの情報
duel_log_dataは、ログの一単位であるDuelLogDataEntryの配列となっています。

DuelLogDataEntryにはtypeとdataのフィールドがあり、typeがログの種類を表し、dataにはそのtypeに応じた補足情報が入っています。

dataは辞書形式ですがそれぞれのdataにはtypeに対応するデータクラスが用意されており、typeに応じたクラスにキャストしてアクセスすることができます。

typeに応じたdataのキャストについては、下記で使用している`text_util.get_duel_log_entry_text(entry)`を参照してください。

In [ ]:
# デュエルログ全体について見てみます
for entry in duel_log_data:
    print(entry) # そのままデータを表示します。
    # typeに応じたtextを出力するget_duel_log_entry_textでテキスト化します。
    text = text_util.get_duel_log_entry_text(entry)
    print(text)


下記ではデータ全体からカードの効果が発動/適用されたログを抽出し、そのcard_idを調べています。

In [ ]:
# 試しにカードの発動のログだけ追ってみます
for entry in duel_log_data:
    print(text_util.get_duel_log_entry_text(entry)) # 
    if entry.type == DuelLogType.CARD_HAPPEN: # 発動のDuelLogTypeはCARD_HAPPEN
        card_happen_data = CardHappenData(entry.data) # CARD_HAPPENに対応する
        print(f"発動/適用したカード: {text_util.get_card_name(card_happen_data.card_id)}") # 例えば発動したカードが取得できます

### デュエル終了時の情報

デュエル終了時の情報もDuelLogDataの一種で、DuelEndDataというデータクラスになっています。

今回は1デュエル分のサンプルデータだったのでログの末尾をDuelEndDataのインスタンスduel_end_dataとして取得しますが、udi_ioからはデュエル終了時に`udi_io.get_duel_end_data()`でも取得できます。

In [ ]:
# ログの末尾を取得
last_log_data = duel_log_data[-1]
if last_log_data.type == DuelLogType.DUEL_END:
    duel_end_data = DuelEndData(entry.data)
    print(duel_end_data)
    print(f"勝敗={duel_end_data.result_type}")